Importing Statements

In [230]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import plotly.express as px
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Dropout, Input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.ticker as mtick

Trading Strategy Code for 1, 2, and 3

In [231]:
def trading_strategy1(predicted_prices, actual_prices):
    predicted_prices = np.atleast_1d(predicted_prices).flatten()
    actual_prices = np.atleast_1d(actual_prices).flatten()
    min_length = min(len(predicted_prices), len(actual_prices))
    predicted_prices = predicted_prices[:min_length]
    actual_prices = actual_prices[:min_length]
    signals = np.zeros_like(predicted_prices, dtype=int)
    buy_mask = predicted_prices > actual_prices
    signals[buy_mask] = 1
    sell_mask = predicted_prices < actual_prices
    signals[sell_mask] = -1
    
    return signals

In [232]:
def trading_strategy2(predicted_prices, actual_prices):
    predicted_prices = np.atleast_1d(predicted_prices).flatten()
    actual_prices = np.atleast_1d(actual_prices).flatten()
    min_length = min(len(predicted_prices), len(actual_prices))
    predicted_prices = predicted_prices[:min_length]
    actual_prices = actual_prices[:min_length]

    signals = np.zeros_like(predicted_prices, dtype=int)
    window_short = 5  
    window_long = 15  

    ma_short = np.zeros_like(predicted_prices)
    ma_long = np.zeros_like(predicted_prices)
    
    for i in range(len(predicted_prices)):
        start_short = max(0, i - window_short + 1)
        start_long = max(0, i - window_long + 1)
    
        ma_short[i] = np.mean(predicted_prices[start_short:i+1])
        ma_long[i] = np.mean(predicted_prices[start_long:i+1])

    for i in range(window_long + 1, len(predicted_prices)):
        buy_conditions = [ma_short[i] > ma_long[i], predicted_prices[i] > predicted_prices[i-1], 
        predicted_prices[i] < ma_short[i] * 0.95, ma_short[i] - ma_short[i-1] > 0]

        sell_conditions = [ma_long[i] > ma_short[i], predicted_prices[i] < predicted_prices[i-1], 
        predicted_prices[i] > ma_short[i] * 1.05, ma_short[i] - ma_short[i-1] < 0]
        
        if sum(buy_conditions) >= 3:
            signals[i] = 1
        elif sum(sell_conditions) >= 3:
            signals[i] = -1

    return signals

In [233]:
def trading_strategy3(predicted_prices, current_prices):
    predicted_prices = np.atleast_1d(predicted_prices).flatten()
    current_prices = np.atleast_1d(current_prices).flatten()

    signals = np.zeros_like(predicted_prices, dtype=int)
    min_length = min(len(predicted_prices), len(current_prices))
    predicted_prices = predicted_prices[:min_length]
    current_prices = current_prices[:min_length]
    buy_mask = predicted_prices > (current_prices * 1.04)
    signals[buy_mask] = 1
    sell_mask = predicted_prices < (current_prices * 0.96)
    signals[sell_mask] = -1
    
    return signals

For the following classifiers, in order to test each strategy, simply change the notated line to whatever strategy you want to test. This is easier than having 12 seperate cells - the outputs are compiled in the tables below this section.

RANDOMFORESTTIMESERIES Classifier


In [ ]:
def optimize_crypto_profits(cryptos=["BTC-USD", "ETH-USD", "ADA-USD"], start_date="2020-01-01", end_date=None, train_ratio=0.7, test_ratio=0.3, features=['Open', 'High', 'Low', 'Close', 'Volume'], target='Close', prediction_horizon=1):
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')
    
    data_dfs = {}
    results = {}
    
    for crypto in cryptos:
        df = yf.download(crypto, start=start_date, end=end_date)
        total_len = len(df)
        train_end = int(total_len * train_ratio)
        test_end = train_end + int(total_len * test_ratio)
        train = df.iloc[:train_end]
        test = df.iloc[train_end:test_end]
        later_test = df.iloc[test_end:]

        def prepare_sequence_data(data, features, target, horizon):
            X = []
            y = []
            for i in range(len(data) - horizon):
                X.append(data[features].iloc[i].values)
                y.append(data[target].iloc[i + horizon])
            return np.array(X), np.array(y)
        
        X_train, y_train = prepare_sequence_data(train, features, target, prediction_horizon)
        
        X_test, y_test = prepare_sequence_data(test, features, target, prediction_horizon)
        
        min_len = min(len(X_test), len(y_test))
        X_test = X_test[:min_len]
        y_test = y_test[:min_len]
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        rf_model = RandomForestRegressor(n_estimators=1000, random_state=8, max_depth=10)
        rf_model.fit(X_train_scaled, y_train)

        y_pred = rf_model.predict(X_test_scaled)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        
        trading_signals = np.where(y_pred > y_test, 1, -1) #THIS LINE

        price_changes = y_test[1:] - y_test[:-1]
        trading_signals = trading_signals[:-1]
        profits = trading_signals * price_changes
        
        results[crypto] = {
            'model': rf_model,
            'scaler': scaler,
            'metrics': {
                'MSE': mse,
                'MAE': mae,
                'Total Profit': np.sum(profits),
                'Average Daily Profit': np.mean(profits),
                'Profit Standard Deviation': np.std(profits)
            },
            'predictions': y_pred,
            'actual_values': y_test,
            'trading_signals': trading_signals
        }
    
    return results

def main():
    crypto_results = optimize_crypto_profits()
    for crypto, result in crypto_results.items():
        print("\nResults for " + str(crypto) + ":")
        print("Performance Metrics:")
        for metric_name, metric_value in result['metrics'].items():
            print(str(metric_name) + ":")
            if isinstance(metric_value, (int, float)):
                print("  " + str(metric_value))
            else:
                print("  " + str(metric_value))
        print("\n")

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
c:\Users\christian fink\anaconda3\envs\skmob\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

[*********************100%***********************]  1 of 1 completed
c:\Users\christian fink\anaconda3\envs\skmob\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

[*********************100%***********************]  1 of 1 completed
c:\Users\christian fink\anaconda3\envs\skmob\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().




Results for BTC-USD:
Performance Metrics:
MSE: 74577361.78102346
MAE: 3700.6894905246827
Total Profit: -270584.6328125
Average Daily Profit: -0.9227978555923498
Profit Standard Deviation: 1530.9409248184702



Results for ETH-USD:
Performance Metrics:
MSE: 10200.589420627153
MAE: 69.08208630619707
Total Profit: 627337.9458007812
Average Daily Profit: 2.1394641118360194
Profit Standard Deviation: 86.7447626007243



Results for ADA-USD:
Performance Metrics:
MSE: 0.0007795083653067362
MAE: 0.017538445431110244
Total Profit: -81.47379469871521
Average Daily Profit: -0.000277857032210118
Profit Standard Deviation: 0.02353423639907463




MLP Classifier 

In [ ]:
def optimize_crypto_profits(cryptos=["BTC-USD", "ETH-USD", "ADA-USD"], start_date="2020-01-01", end_date=None, train_ratio=0.7, test_ratio=0.3, features=['Open', 'High', 'Low', 'Close', 'Volume'], target='Close', prediction_horizon=1):
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')
    
    data_dfs = {}
    results = {}
    
    for crypto in cryptos:
        df = yf.download(crypto, start=start_date, end=end_date)
        total_len = len(df)
        train_end = int(total_len * train_ratio)
        test_end = train_end + int(total_len * test_ratio)
        train = df.iloc[:train_end]
        test = df.iloc[train_end:test_end]
        later_test = df.iloc[test_end:]

        def prepare_sequence_data(data, features, target, horizon):
            X = []
            y = []
            for i in range(len(data) - horizon):
                X.append(data[features].iloc[i].values)
                price_change = data[target].iloc[i + horizon].item() - data[target].iloc[i].item()
                y.append(1 if price_change > 0 else 0)
            return np.array(X), np.array(y)

        X_train, y_train = prepare_sequence_data(train, features, target, prediction_horizon)
        
        X_test, y_test = prepare_sequence_data(test, features, target, prediction_horizon)

        min_len = min(len(X_test), len(y_test))
        X_test = X_test[:min_len]
        y_test = y_test[:min_len]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        mlp_model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=8, learning_rate_init=0.001, activation='relu')
        mlp_model.fit(X_train_scaled, y_train)

        y_pred = mlp_model.predict(X_test_scaled)

        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
    
        trading_signals = trading_strategy1(y_pred,y_test) #THIS LINE
        
        price_changes = test[target].iloc[prediction_horizon:].values - test[target].iloc[:-prediction_horizon].values

        trading_signals = trading_signals[:len(price_changes)]
        profits = trading_signals * price_changes
        
        results[crypto] = {
            'model': mlp_model,
            'scaler': scaler,
            'metrics': {
                'MSE': mse,
                'MAE': mae,
                'Total Profit': np.sum(profits),
                'Average Daily Profit': np.mean(profits),
                'Profit Standard Deviation': np.std(profits)
            },
            'predictions': y_pred,
            'actual_values': y_test,
            'trading_signals': trading_signals
        }
    
    return results

def main():
    crypto_results = optimize_crypto_profits()
    for crypto, result in crypto_results.items():
        print("\nResults for " + str(crypto) + ":")
        print("Performance Metrics:")
        for metric_name, metric_value in result['metrics'].items():
            print(str(metric_name) + ":")
            if isinstance(metric_value, (int, float)):
                print("  " + str(metric_value))
            else:
                print("  " + str(metric_value))
        print("\n")

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Results for BTC-USD:
Performance Metrics:
MSE:
  0.5018450184501845
MAE:
  0.5018450184501845
Total Profit:
  -9341447.0
Average Daily Profit:
  -31.799155104097167
Profit Standard Deviation:
  1083.0759090218319



Results for ETH-USD:
Performance Metrics:
MSE:
  0.4797047970479705
MAE:
  0.4797047970479705
Total Profit:
  -75406.2802734375
Average Daily Profit:
  -0.25668999698205874
Profit Standard Deviation:
  60.044387137763316



Results for ADA-USD:
Performance Metrics:
MSE:
  0.466789667896679
MAE:
  0.466789667896679
Total Profit:
  -125.96221312880516
Average Daily Profit:
  -0.0004287870982448672
Profit Standard Deviation:
  0.016060741539304473




Strategy 2 for some reason returns all 0s for the MLP Classifier. It continues to have a MSE and MAE (maybe those are calculated in error?), but the Total Profit, Average Daily Profit, and Profit Standard Deviation are all 0.0.

SEQUENTIAL TENSORFLOW Classifier

In [ ]:
def optimize_crypto_profits(cryptos=["BTC-USD", "ETH-USD", "ADA-USD"], start_date="2020-01-01", end_date=None, 
                             train_ratio=0.7, test_ratio=0.3, features=['Open', 'High', 'Low', 'Close', 'Volume'], 
                             target='Close', prediction_horizon=1, lstm_units=50, dropout_rate=0.2):
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')
    
    data_dfs = {}
    results = {}
    
    for crypto in cryptos:
        df = yf.download(crypto, start=start_date, end=end_date)
        total_len = len(df)
        train_end = int(total_len * train_ratio)
        test_end = train_end + int(total_len * test_ratio)
        train = df.iloc[:train_end]
        test = df.iloc[train_end:test_end]
        later_test = df.iloc[test_end:]

        def prepare_sequence_data(data, features, target, horizon):
            X = []
            y = []
            for i in range(len(data) - horizon):
                X.append(data[features].iloc[i:i+horizon].values)
                y.append(data[target].iloc[i + horizon])
            return np.array(X), np.array(y)
        
        X_train, y_train = prepare_sequence_data(train, features, target, prediction_horizon)
        X_test, y_test = prepare_sequence_data(test, features, target, prediction_horizon)
        min_len = min(len(X_test), len(y_test))
        X_test = X_test[:min_len]
        y_test = y_test[:min_len]
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
        X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
        
        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(lstm_units, input_shape=(X_train.shape[1], X_train.shape[2]), 
                                  return_sequences=False),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(1)
        ])
        
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=0)
        y_pred = model.predict(X_test_scaled).flatten()
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        
        trading_signals = trading_strategy1(y_pred,y_test) #THIS LINE

        price_changes = y_test[1:] - y_test[:-1]
        trading_signals = trading_signals[:-1]
        profits = trading_signals * price_changes
        
        results[crypto] = {
            'model': model,
            'scaler': scaler,
            'metrics': {
                'MSE': mse,
                'MAE': mae,
                'Total Profit': np.sum(profits),
                'Average Daily Profit': np.mean(profits),
                'Profit Standard Deviation': np.std(profits)
            },
            'predictions': y_pred,
            'actual_values': y_test,
            'trading_signals': trading_signals
        }
    
    return results

def main():
    crypto_results = optimize_crypto_profits()
    np.random.seed(8)
    tf.random.set_seed(8)
    for crypto, result in crypto_results.items():
        print("\nResults for " + str(crypto) + ":")
        print("Performance Metrics:")
        for metric_name, metric_value in result['metrics'].items():
            print(str(metric_name) + ":")
            if isinstance(metric_value, (int, float)):
                print("  " + str(metric_value))
            else:
                print("  " + str(metric_value))
        print("\n")
        
if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
c:\Users\christian fink\anaconda3\envs\skmob\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step


[*********************100%***********************]  1 of 1 completed
c:\Users\christian fink\anaconda3\envs\skmob\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step


[*********************100%***********************]  1 of 1 completed
c:\Users\christian fink\anaconda3\envs\skmob\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step

Results for BTC-USD:
Performance Metrics:
MSE: 2577657634.3395534
MAE: 47029.961780731086
Total Potential Profit: -40835233.6796875
Average Daily Profit: -139.52130025415897
Profit Standard Deviation: 1524.570357056474



Results for ETH-USD:
Performance Metrics:
MSE: 166339.673697036
MAE: 335.5442031916657
Total Potential Profit: 221699.23278808594
Average Daily Profit: 0.757477365418616
Profit Standard Deviation: 86.76783605335983



Results for ADA-USD:
Performance Metrics:
MSE: 0.0006873780805819663
MAE: 0.015456276461207119
Total Potential Profit: -2.3665319681167603
Average Daily Profit: -8.085704121951068e-06
Profit Standard Deviation: 0.023535875212943896




VISUALIZATION

Calculating Averages to Normalize all Data

In [237]:
cryptos = ["BTC-USD", "ETH-USD", "ADA-USD"]
start_date = "2020-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

data_dfs = {}
for crypto in cryptos:
    data_dfs[crypto] = yf.download(crypto, start=start_date, end=end_date)

btc_df = data_dfs["BTC-USD"]
eth_df = data_dfs["ETH-USD"] 
ada_df = data_dfs["ADA-USD"] 

average_prices = {}
for crypto, df in data_dfs.items():
    avg_price = df['Close'].mean()  
    average_prices[crypto] = avg_price
average_prices

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


{'BTC-USD': Ticker
 BTC-USD    35855.936565
 dtype: float64,
 'ETH-USD': Ticker
 ETH-USD    1971.38041
 dtype: float64,
 'ADA-USD': Ticker
 ADA-USD    0.618607
 dtype: float64}

Average Daily Profit for each strategy using each classification. The values are normalized, in order to better compare them (also, all numbers are *1000 to make it easier to read). Normalization involved dividing each value by the average price of the crypto over the course of the time period.

| **BTC** | Strategy 1 | Strategy 2 | Strategy 3 | Classifier Strategy |
| :---- | :---- | :---- | :---- | :---- |
| RandomForest | \-1.256794664 | 0.3299963565 | \-0.8565863085 | 0.03677401189 |
| MLPClassifier | \-1.180648162 | 0 | \-1.180648162 | \-3.011713277 |
| Sequential | \-3.798468789 | 0.610844333 | \-3.798468789 | \-3.798468787 |
|  |  |  |  |  |
| **ETH** | Strategy 1 | Strategy 2 | Strategy 3 | Classifier Strategy |
| RandomForest | 0.06070029234 | 0.03570602745 | \-0.0035705875 | 1.102372813 |
| MLPClassifier | 0.0538777965 | 0 | \-0.0538777965 | \-1.070372731 |
| Sequential | 0.4820315482 | 0.02142362662 | 0.3177838118 | 0.2803775529 |
|  |  |  |  |  |
| **ADA** | Strategy 1 | Strategy 2 | Strategy 3 | Classifier Strategy |
| RandomForest | \-0.0841935484 | \-0.0266129032 | \-0.0932258065 | \-0.540483871 |
| MLPClassifier | \-0.8998387097 | 0 | \-0.9 | \-2.138387097 |
| Sequential | \-1.317903226 | 0.09758064516 | \-0.1908064516 | \-0.5717741935 |
|  |  |  |  |  |
| **Total Profit** | \-7.94123116 | 1.06893809 | \-6.75940009 | \-9.71167558 |

Profit Standard Deviation (Every number is *1000 for consistency)

| **BTC** | Strategy 1 | Strategy 2 | Strategy 3 | Classifier Strategy |
| :---- | ----- | ----- | ----- | ----- |
| RandomForest | 42.67457114 | 25.36522538 | 23.7756148 | 42.69305801 |
| MLPClassifier | 30.80608705 | 0 | 30.80608705 | 42.55757737 |
| Sequential | 42.5237603 | 27.83016013 | 42.5237603 | 42.5237603 |
|  |  |  |  |  |
| **ETH** | Strategy 1 | Strategy 2 | Strategy 3 | Classifier Strategy |
| RandomForest | 44.03376765 | 28.01622912 | 19.94569202 | 44.02000854 |
| MLPClassifier | 30.41276887 | 0 | 30.41276887 | 43.98228961 |
| Sequential | 44.03196093 | 27.36909709 | 41.94727856 | 44.03291685 |
|  |  |  |  |  |
| **ADA** | Strategy 1 | Strategy 2 | Strategy 3 | Classifier Strategy |
| RandomForest | 37.93056129 | 23.96725645 | 21.19975629 | 37.92680365 |
| MLPClassifier | 26.4342969 | 0 | 26.43429677 | 37.83768226 |
| Sequential | 37.92992615 | 24.83889952 | 19.50064274 | 37.92633871 |
|  |  |  |  |  |
| **Average Standard Deviation** | 37.41974447 | 17.48742974 | 28.50509971 | 41.50004837 |

These are graphs of each crypto over the course of 2024, also graphed with them are the FMA and SMA (5-day and 15-day periods, respectively).

In [238]:
btc_df.columns = btc_df.columns.droplevel(1) 
btc_df.reset_index(inplace=True)
btc_df = btc_df.loc[btc_df['Date'].dt.year == 2024]
btc_df['MA5'] = btc_df['Close'].rolling(window=5).mean()
btc_df['MA15'] = btc_df['Close'].rolling(window=15).mean()

fig = px.line(btc_df, x="Date", y=['Close', 'MA5', 'MA15'], title="BTC-USD Training Data: Close Prices Over Time", labels={'value': 'Price in USD', 'variable': 'Legend'})
fig.show()

C:\Users\christian fink\AppData\Local\Temp\ipykernel_14572\2156963694.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\christian fink\AppData\Local\Temp\ipykernel_14572\2156963694.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [239]:
eth_df.columns = eth_df.columns.droplevel(1) 
eth_df.reset_index(inplace=True)
eth_df = eth_df.loc[eth_df['Date'].dt.year == 2024]
eth_df['MA5'] = eth_df['Close'].rolling(window=5).mean()
eth_df['MA15'] = eth_df['Close'].rolling(window=15).mean()

fig = px.line(eth_df, x="Date", y=['Close', 'MA5', 'MA15'], title="ETH-USD Training Data: Close Prices Over Time", labels={'value': 'Price in USD', 'variable': 'Legend'})

fig.show()

C:\Users\christian fink\AppData\Local\Temp\ipykernel_14572\230048122.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\christian fink\AppData\Local\Temp\ipykernel_14572\230048122.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [240]:
ada_df.columns = ada_df.columns.droplevel(1) 
ada_df.reset_index(inplace=True)
ada_df = ada_df.loc[ada_df['Date'].dt.year == 2024]
ada_df['MA5'] = ada_df['Close'].rolling(window=5).mean()
ada_df['MA15'] = ada_df['Close'].rolling(window=15).mean()

fig = px.line(ada_df, x="Date", y=['Close', 'MA5', 'MA15'], title="ADA-USD Training Data: Close Prices Over Time", labels={'value': 'Price in USD', 'variable': 'Legend'})

fig.show()

C:\Users\christian fink\AppData\Local\Temp\ipykernel_14572\3553793140.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\christian fink\AppData\Local\Temp\ipykernel_14572\3553793140.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

